In [1]:
import pandas as pd
import numpy as np

In [2]:
import os

In [3]:
PATH = '..\..'
os.listdir(PATH)
os.chdir(PATH)

In [4]:
files = ["transactions_log_201702", "transactions_log_201701", "transactions_log_201612", 
         "transactions_log_201611", "transactions_log_201610", "transactions_log_201609"]
    
#combining transactions log
df0 = pd.concat([pd.read_csv(f + '.csv') for f in files])
df0 = df0.sort_values(['msno','transaction_date','membership_expire_date'])
    
#Aggregating transactions log for feature creation
#aggregating columns for each msno
df1 = df0.groupby('msno').agg({'payment_plan_days':['mean'], 'actual_amount_paid':['mean'], 
                               'plan_list_price':['mean'],'payment_method_id':'count'}).reset_index()

In [5]:
df1.columns = ['msno', 'payment_plan_days_mean','actual_amount_paid_mean','plan_list_price_mean','payment_counts']

df1.columns = ['msno'] + [str(col) for col in df1.columns[1:]]

In [6]:
# length of df1
len(df1)

1520524

In [7]:
df0 = df0.reset_index(drop = True)

In [8]:
#Getting the latest record for each msno
df2 = df0.loc[df0.groupby('msno').transaction_date.idxmax()].reset_index(drop=True)
    
df2.columns = ['msno','latest_payment_method_id','latest_payment_plan_days',
               'latest_plan_list_price_mean','latest_actual_amount_paid','latest_is_auto_renew',
               'latest_trnsaction_date','latest_membership_expire_date','latest_is_cancel']

df2.columns = ['msno'] + [str(col) for col in df2.columns[1:]]

In [9]:
len(df2)

1520524

In [10]:
#merging the two dataframes
df = pd.merge(df1, df2, how='inner', left_on=['msno'], right_on=['msno'])

In [11]:
len(df)

1520524

In [12]:
#writing the combined aggregated transaction_logs to csv
df.to_csv("transaction_logs_initial.csv", index = False)